# Multi-output linear classification

In the previous notebook, we have seen a simple neural network regressor (Linear regression) and classifier (Logistic regression) that were implemented as a single artificial neuron.

In this notebook we will implement a __multi-output single layer perceptron__ to obtain a multi-label linear classification model

## Multi-label linear classification
<img src="images/softmax_cropped.png" width = "300" style="float: right;">

A single layer perceptron consists of multiple neurons organised in one layer. These  neurons  share  the  same  input  features,  but  each  of  them  produces a different output. The outputs of the linear layer for each neuron $z_k=\sum_jw_{jk}x_j$ are either passed through the same activation function $f(z)$, or in case of multi-label classification through a shared **softmax** activation function:
$$ \hat{p}_k=\frac{e^z_k}{\sum_{j=1}^Ke^z_{j}}$$

The linear multi-label classification is implemented with a **single linear layer**, with
* the number of **inputs** equal to the number of **features**
* the number of **outputs** equal to the number of **classes**

For example, if we would like to predict no, mild moderate and severe heart failure from EF and GLS, we need two inputs and three outputs, as implemented in the cell below.

In [ ]:
import torch
from torch import nn
l = nn.Linear(2,3)
print(l)

The loss will be set to **cross-entropy** using the in-built function `CrossEntropyLoss`. This function combines softmax with cross-entropy loss, so we will not need to implement the activation function in our network. For numerical reasons, Pytorch implements log-softmax followed by negative log-likelihood loss in this function.

In [ ]:
loss_function = nn.CrossEntropyLoss()

## Exercise 2: Multi-label linear classifier

In this exercise we will implement a multi-label classifier in Pytorch, to predict no, mild moderate and severe heart failure from EF and GLS. Code below loads and plots the data, and converts the data into Pytorch tensors.

Note, that the input features are required to be of type `float`, while output labels need to be of type `long`.

In [ ]:
# only do this if you work on Google Colab
# run the cell
# then upload file 'heart_failure_data_complete.csv'

from google.colab import files
files.upload()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('heart_failure_data_complete.csv')
data = df.to_numpy()
X = data[:,[1,2]]
X = StandardScaler().fit_transform(X)
y = data[:,0]

def PlotData(X,y,fontsize=12):
    plt.plot(X[y==0,0],X[y==0,1],'bo',alpha=0.75,markeredgecolor='k',label = 'Healthy')
    plt.plot(X[y==1,0],X[y==1,1],'rd',alpha=0.75,markeredgecolor='k',label = 'moderate HF')
    plt.plot(X[y==2,0],X[y==2,1],'g^',alpha=0.75,markeredgecolor='k',label = 'severe HF')
    plt.title('Diagnosis of Heart Failure', fontsize = fontsize+2)
    plt.xlabel('EF', fontsize = fontsize)
    plt.ylabel('GLS', fontsize = fontsize)
    plt.legend(fontsize = fontsize-2)

PlotData(X,y)

# convert numpy array to tensor in shape of input size
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).long()
print('X: ', X.shape)
print('y: ', y.shape)

Below is the function to plot the classification result. Run the code.

In [ ]:
def PlotClassification(net,X,y,fontsize=12):

    # Create an 1D array of samples for each feature
    x1 = np.linspace(-2.5, 2, 1000) 
    x2 = np.linspace(-3, 3.5, 1000).T # note the transpose
    # Creates 2D arrays that hold the coordinates in 2D feature space
    x1, x2 = np.meshgrid(x1, x2) 
    # Flatten x1 and x2 to 1D vector and concatenate into a feature matrix
    Feature_space = np.c_[x1.ravel(), x2.ravel()] 
    
    # NEW: convert numpy to torch
    Feature_space = torch.from_numpy(Feature_space).float()
    # NEW: Predict output scores for the whole feature space    
    output_scores = net(Feature_space)
    # NEW: Take maximum to get the labels
    _,y_pred=torch.max(output_scores, 1)
    # NEW: Convert to numpy
    y_pred = y_pred.numpy()
    
    # Resahpe to 2D
    y_pred = y_pred.reshape(x1.shape)
    # Plot using contourf
    plt.contourf(x1, x2, y_pred, cmap = 'summer')
    
    # Plot data
    PlotData(X,y,fontsize)

### Train test split
**Task 2.1:** First, split the data into training set and test set. For this, we will use scikit-learn `train_test_split`. Note that this function works on Pytorch tensors the same way as on numpy arrays. Use 33% of the data for testing. Note the types of the split dataset.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = None

print('Test features type:', X_test.type())
print('Test labels type:', y_test.type())
print('Test labels:', y_test)

### Create and train the network
**Task 2.2:** Fill in the code below to create and train multi-label classification model in Pytorch. Make sure that the network is trained using only training data

In [ ]:
# network architecture
class MultiLabelClassifier(nn.Module):
    def __init__(self):
        super(MultiLabelClassifier, self).__init__()
        self.layer = None

    def forward(self, x):
        x = None 
        return x

# create model    
net = MultiLabelClassifier()

# loss
loss_function = None

# optimiser
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)

# train for 500 epochs
epochs = 500
for i in range(epochs):
    optimizer.zero_grad() 
    prediction = None 
    loss = loss_function(prediction, None) 
    loss.backward()       
    optimizer.step() 

# Plot result
PlotClassification(net,None,None)

### Evaluate training accuracy

We will now show how we can predict the labels on the test set using this network. Because softmax and cross-entropy loss are combined, for each sample the network will return three outputs of the linear layer $z_0,z_1,z_2$ that correspond to the three classes. These outputs are referred to as **logits**. Let's test that for an individual feature vector $x=(0,0)$ that we create:

In [ ]:
# create a feature vector of correct shape and type
x = torch.tensor((0,0)).reshape(1,2).float()
# predict using forward pass
z = net(x)
# print logits
print('Logits: ', z)

To find the label for this datapoint, we need to find which class returned the largest logit:

In [ ]:
y = torch.argmax(z, dim=1)
print('Predicted label: ', y)

**Task 2.3:** Fill in the code to predict the labels for the training set

In [ ]:
# forward pass
pred=None
# find maximum
y_pred_train = None
print(y_pred_train)

Pytorch does not offer functions for calculating performance measures, by the can use `accuracy_score` from scikit-learn.

In [ ]:
from sklearn.metrics import accuracy_score
print('Training accuracy: ', accuracy_score(y_train, y_pred_train))

### Evaluate on test set

**Task 2.3:** To evaluate accuracy on the test set, implement the following:
* predict the logits for the test set by running a forward pass through the network
* convert logits to label
* calculate test accuracy score
* plot the classification result for the test set

In [ ]:
# forward pass
pred=None

# find maximum
y_pred_test = None

# calculate accuracy
print('Test accuracy: ', None)

# plot
